In [39]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# **SUI DATASET**

Entrambi i dataset sono versioni pre-processate del dataset **OASIS Alzheimer's Detection** (un dataset di immagini MRI del cervello di 461 pazienti ognuno dei quali ha tre scansioni MRI) utilizzato per rilevare i segni precoci dell’Alzheimer.

Poiché le immagini MRI sono 3D, sono state divise lungo l'asse z (l'asse della profondità del cervello) in 256 fette e sono state selezionate le fette che vanno dalla 100 esima alla 160 esima al fine di ridurre il numero di immagini e concentrarsi sulle sezioni cerebrali rilevanti per l'analisi della demenza; Questo approccio ha generato immagini 2D per ogni paziente.


****

Le immagini sono classificate in **quattro sottoclassi**, in base alla progressione della malattia:

*   Moderate dementia (affetto da demenza)

*   Very mild dementia (affetto da demenza molto lieve)
* Mild dementia (affetto da demenza lieve)
* Non-demented (non affetto da demenza)


*La classificazione dei pazienti è stata effettuata sulla base delle valutazioni CDR (Clinical Dementia Rating), un punteggio che valuta la severità della demenza.*


In [40]:
#estrazione db 1
import zipfile
import os

zip_path = "/content/drive/My Drive/Alzheimer-detection-project/oasis ds.zip"
extract_folder = "/content/Alzheimer-detection-project"

os.makedirs(extract_folder, exist_ok=True)
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_folder)

print("File estratti:", os.listdir(extract_folder))


File estratti: ['oasis ds', '__MACOSX', 'archive']


In [33]:
# numero di immagini per categoria db 1
dementia_categories = ['Moderate Dementia', 'Mild Dementia', 'Very mild Dementia', 'Non Demented']
base_path = os.path.join(extract_folder, 'oasis ds')


for category in dementia_categories:
    category_path = os.path.join(base_path, category)
    images = os.listdir(category_path)
    print(f"{category}: {len(images)} immagini")


Moderate Dementia: 98 immagini
Mild Dementia: 1001 immagini
Very mild Dementia: 2745 immagini
Non Demented: 13445 immagini


In [42]:
#estrazione db 2
zip_path = "/content/drive/My Drive/Alzheimer-detection-project/archive.zip"
extract_folder = "/content/Alzheimer-detection-project"

os.makedirs(extract_folder, exist_ok=True)
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_folder)

print("File estratti:", os.listdir(extract_folder))


File estratti: ['oasis ds', '__MACOSX', 'archive']


In [36]:
# numero di immagini per categoria db 2
dementia_categories = ['Moderate Dementia', 'Mild Dementia', 'Very mild Dementia', 'Non Demented']
base_path = os.path.join(extract_folder, 'archive')


for category in dementia_categories:
    category_path = os.path.join(base_path, category)
    images = os.listdir(category_path)
    print(f"{category}: {len(images)} immagini")


Moderate Dementia: 376 immagini
Mild Dementia: 5184 immagini
Very mild Dementia: 13797 immagini
Non Demented: 63560 immagini
